In [ ]:
import pandas as pd
from pulp import *
import math 
import numpy as np
import xlrd
from array import *
import json
from flask import Flask, redirect, url_for, request, render_template, session
import urllib
import os.path
from os import path


In [ ]:
app = Flask(__name__)
app.secret_key = 'aqswdefrgt'
sessionVariable = 0

In [ ]:
@app.route('/login',methods = ["POST"])
def login():
    username = request.form['username']
    password = request.form['password']
    data = {}
    if(username=="admin@iit.com" and password=="usn@321###"):
        data['status'] = 1
        session['username'] = username
    else:
        data['status'] = 0

    json_data = json.dumps(data)
    json_object = json.loads(json_data)
    return(json.dumps(json_object, indent = 1))

In [ ]:
# log-out feature
@app.route("/logout")
# @login_required
def logout():
    session.pop('username', None)
    return render_template("login.html")

# Cache clear
@app.after_request
def after_request(response):
    response.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
    return response

In [ ]:
@app.route("/uploadConfigExcel",methods = ["POST"])
def uploadConfigExcel():
    data = {}
    try:
        file = request.files['uploadFile']
        if os.path.exists("Data_1.xlsx"):
            os.remove("Data_1.xlsx")
            file.save('Data_1.xlsx')
        
        data['status'] = 1
    except:
        data['status'] = 0
    
    json_data = json.dumps(data)
    json_object = json.loads(json_data)

    return(json.dumps(json_object, indent = 1))



In [ ]:
@app.route("/processFile",methods = ["POST"])
def processFile():
    if 'username' not in session:
        return render_template("login.html")
    try:
        USN = pd.ExcelFile("Data_1.xlsx")
    except:
        data = {}
        data["status"] = 0
        data["message"] = message
        json_data = json.dumps(data)
        json_object = json.loads(json_data) 
        return(json.dumps(json_object, indent = 1))


    message = "Cost matrix file is incorrect"
    try:
        WKB = xlrd.open_workbook("USN_LastLeg1_Distance.xlsx")
    except:
        data = {}
        data["status"] = 0
        data["message"] = message
        json_data = json.dumps(data)
        json_object = json.loads(json_data) 
        return(json.dumps(json_object, indent = 1))

        Sheet1 = WKB.sheet_by_index(0)
        FCI=pd.read_excel(USN,sheet_name='FCI', index_col=None)
        FPS=pd.read_excel(USN,sheet_name='FPS',index_col=None)
        Original_Tagging=pd.read_excel(USN,sheet_name='Original_Tagging_FCI_FPS',index_col=None)

        Warehouse_No=[]
        FPS_No=[]
        Warehouse_No=FCI['FCI_ID'].nunique()
        FPS_No=FPS['FPS_ID'].nunique()
        Warehouse_Count={}
        FPS_Count={}
        Warehouse_Count["Warehouse_Count"]=Warehouse_No
        FPS_Count["FPS_Count"]=FPS_No
        
        FCI_district = []
        FCI_Data = {}
        Disrticts_FCI = {}
        Data_state_wise = {}
        Data_statewise = {}

        for i,j in zip(FCI["FCI_District"],FCI["FCI_ID"]):
            i = i.lower()
            if i not in FCI_district:
                FCI_district.append(i)
                globals()[f"FCI_{i}"] = []
            globals()[f"FCI_{i}"].append(j)
        for i in FCI_district:
            FCI_Data[i] = globals()[f"FCI_{i}"]
        Disrticts_FCI["Disrticts_FCI"] = FCI_district

        FPS_district = []
        FPS_Data = {}
        Districts_FPS = {}
        for i,j in zip(FPS["FPS_District"],FPS["FPS_Tehsil"]):
            i = i.lower()
            if i not in FPS_district:
                FPS_district.append(i)
                globals()[f"FPS_{i}"] = []
            if j not in globals()[f"FPS_{i}"]:
                globals()[f"FPS_{i}"].append(j)
        for i in FPS_district:
            FPS_Data[i] = globals()[f"FPS_{i}"]
        Districts_FPS["Districts_FPS"] = FPS_district


        
        model = LpProblem('Supply-Demand-Problem', LpMinimize)

        Variable1=[]
        Variable2=[]
        for i in range(len(FCI["FCI_ID"])):
            for j in range(len(FPS["FPS_ID"])):
                Variable1.append(str(FCI["FCI_ID"][i]) + "_" + str(FCI["FCI_District"][i])+"_"+ str(FPS["FPS_ID"][j])+ "_"+ str(FPS["FPS_District"][j]) + "_Wheat")
                Variable2.append(str(FCI["FCI_ID"][i]) + "_" + str(FCI["FCI_District"][i])+"_"+ str(FPS["FPS_ID"][j])+ "_"+ str(FPS["FPS_District"][j]) + "_Rice")

        #Variables for Wheat from lEVEL2 TO FPS 
        DV_Variables1 = LpVariable.matrix("X",Variable1,cat="float",lowBound=0)
        Allocation1 = np.array(DV_Variables1).reshape(len(FCI["FCI_ID"]),len(FPS["FPS_ID"]))

        #Variables for rice from Level2 TO FPS
        DV_Variables2 = LpVariable.matrix("Y",Variable2,cat="float",lowBound=0)
        Allocation2 = np.array(DV_Variables2).reshape(len(FCI["FCI_ID"]),len(FPS["FPS_ID"]))
        
        Tehsil = {}
        UniqueId = 0
        Tehsil_temp = []
        Tehsil_rev = {}
        
        for i in FPS["FPS_Tehsil"]:
            Tehsil_temp.append(i)
            if i not in Tehsil:
                Tehsil[i] = UniqueId
                Tehsil_rev[UniqueId] = i
                UniqueId = UniqueId + 1
                
        Tehsil_FPS = []
        for i in range(len(FPS["FPS_ID"])):
            Tehsil_FPS.append(Tehsil[Tehsil_temp[i]])
                    
                
        
        FCI_FPS=[]
        for col in range(Sheet1.nrows):
            temp = []
            for row in range (Sheet1.ncols):
                temp.append(Sheet1.cell_value(col,row))
            FCI_FPS.append(temp)
            
            
        allCombination1 = []
        allCombination2 = []

        for i in range(len(FCI_FPS)):
            for j in range(len(FPS["FPS_ID"])):
                allCombination1.append(Allocation1[i][j]*FCI_FPS[i][j])
                
        for i in range(len(FCI_FPS)):
            for j in range(len(FPS["FPS_ID"])):
                allCombination2.append(Allocation2[i][j]*FCI_FPS[i][j])

        model += lpSum(allCombination1+allCombination2)

        
        #Demand Constraints for Wheat
        for i in range(len(FPS["FPS_ID"])):
            #print((lpSum(Allocation1[j][i] for j in range(len(FCI["FCI_Code"]))))>=FPS["Allocation_Wheat"][i])
            model+=((lpSum(Allocation1[j][i] for j in range(len(FCI["FCI_ID"]))))>=FPS["Allocation_Wheat"][i])


        #Demand Constraints for Rice
        for i in range(len(FPS["FPS_ID"])):
            #print((lpSum(Allocation2[j][i] for j in range(len(FCI["FCI_Code"]))))>=FPS["Rice_Allocation"][i])
            model+=((lpSum(Allocation2[j][i] for j in range(len(FCI["FCI_ID"]))))>=FPS["Allocation_Rice"][i])

            
        #Supply Constraints for Warehouses
        for i in range(len(FCI["FCI_ID"])):
            #print(lpSum(Allocation1[i][j] for j in range(len(FPS["FPS_Code"])))+(lpSum(Allocation2[i][k] for k in range(len(FPS["FPS_Code"]))))<=FCI["FCI_Capacity"][i])
            model+=(lpSum(Allocation1[i][j] for j in range(len(FPS["FPS_ID"])))+(lpSum(Allocation2[i][k] for k in range(len(FPS["FPS_ID"]))))<=FCI["FCI_Capacity"][i])


        #Calling CBC_CMB Solver
        model.solve(PULP_CBC_CMD())
        Status = LpStatus[model.status]
        
        Original_Cost=100000000
        total =Original_Cost

        data = {}
        data["status"] = 1
        data["modelStatus"] = Status
        data["totalCost"] = float(round(model.objective.value(), 2))
        data["original"] = float(round(total, 2))
        data["percentageReduction"] = float(round(((total - model.objective.value())/total), 4)*100)
        
        
        BGW = {}
        BGR = {}
        IGW = {}
        IGR = {}
        FCIW = {}

        BGCapacity = {}

        temp = {}
        for i in range(len(FCI["FCI_ID"])):
            temp[str(FCI["FCI_ID"][i])] = str(FCI["FCI_Capacity"])
        BGCapacity = temp
        
        temp1 = {}
        BG_FPS = [[] for i in range(len(Tehsil))]
        for i in range(len(FCI["FCI_ID"])):
            for j in range(len(FPS['FPS_ID'])):
                BG_FPS[Tehsil_FPS[j]].append(Allocation1[i][j].value())
            temp1[str(FCI["FCI_ID"][i])] = str(lpSum(Allocation1[i][j].value() for j in range(len(FPS["FPS_ID"]))))
            BGCapacity[str(FCI["FCI_ID"][i])] = str(FCI["FCI_Capacity"][i])
        BGW["FPS"] = temp1

        
        BG_FPS_Wheat = {}
        for i in range(len(Tehsil)):
            BG_FPS_Wheat[str(Tehsil_rev[i])] = str(lpSum(BG_FPS[i]))
            
        temp2 = {}
        BG_FPS = [[] for i in range(len(Tehsil))]
        for i in range(len(FCI["FCI_ID"])):
            for j in range(len(FPS['FPS_ID'])):
                BG_FPS[Tehsil_FPS[j]].append(Allocation2[i][j].value())
            temp2[str(FCI["FCI_ID"][i])] = str(lpSum(Allocation2[i][j].value() for j in range(len(FPS["FPS_ID"]))))
        BGR["FPS"] = temp2

        BG_FPS_Rice = {}
        for i in range(len(Tehsil)):
            BG_FPS_Rice[str(Tehsil_rev[i])] = str(lpSum(BG_FPS[i]))
            
                    
        data["BGW"] = BGW
        data["BGR"] = BGR
        data["FPSW"] = BG_FPS_Wheat
        data["FPSR"] = BG_FPS_Rice
        data["BGCapacity"] = BGCapacity
        
        
        Output_File = open("output.csv","w")
        #Writing Values in output file
        for v in model.variables():
            Output_File.write(v.name + "\t" + str(v.value()) + "\n")
        
        Output_File.close()
        Output_File = open("output.csv","r")
        count_BG_FPS_Wheat = 0
        count_BG_FPS_Rice = 0
        
        for line in Output_File:
            score = float(line.split("\t")[1])
            if(score>0.0):
                tag = line.split("\t")[0]
                if(tag in BG_FPS_Wheat):
                    count_BG_FPS_Wheat = count_BG_FPS_Wheat + 1
                if(tag in BG_FPS_Rice):
                    count_BG_FPS_Rice = count_BG_FPS_Rice + 1

        tableData = {}
        optimalData = {}
        optimalData["count_BG_FPS_Wheat"] = str(count_BG_FPS_Wheat)
        optimalData["count_BG_FPS_Rice"] = str(count_BG_FPS_Rice)
        tableData["optimalData"] = optimalData

        
        Original_Tagging_Wheat_BG_FPS = len(Original_Tagging["FPS_ID"])
        Original_Tagging_Rice_BG_FPS = len(Original_Tagging["FPS_ID"])

        
        
        originalData = {}
        originalData["Original_Tagging_Wheat_BG_FPS"] = str(Original_Tagging_Wheat_BG_FPS)
        originalData["Original_Tagging_Rice_BG_FPS"] = str(Original_Tagging_Rice_BG_FPS)
        tableData["originalData"] = originalData
        

        Original_BG_FPS_Wheat = []
        Original_BG_FPS_Rice = []
        

        Original_BG_FPS_Rice = Original_Tagging["FPS_ID"]
        Sum_BG_FPS_Rice = 0
        for i in range(0, len(Original_BG_FPS_Rice)):
            Sum_BG_FPS_Rice = (str(Sum_BG_FPS_Rice)) + (str(Original_BG_FPS_Rice[i]))

        Original_BG_FPS_Wheat = Original_Tagging["FPS_ID"]
        Sum_BG_FPS_Wheat = 0
        for i in range(0, len(Original_BG_FPS_Wheat)):
            Sum_BG_FPS_Wheat = (str(Sum_BG_FPS_Wheat))+ (str(Original_BG_FPS_Wheat[i]))


            
        sumData = {}
        sumData["Sum_BG_FPS_Rice"] = str(Sum_BG_FPS_Rice)
        sumData["Sum_BG_FPS_Wheat"] = str(Sum_BG_FPS_Wheat)
        tableData["sumData"] = sumData
        
        
        Data_state_wise["FCI_Data"] = FCI_Data
        Data_state_wise["FPS_Data"] = FPS_Data
        Data_state_wise["FCI_district"]= FCI_district
        Data_state_wise["FPS_district"]= FPS_district

    
        data["TableData"] = tableData
        data["Datastatewise"] = Data_state_wise
        
        json_data = json.dumps(data)
        json_object = json.loads(json_data)

        if os.path.exists("ouputPickle.pkl"):
            os.remove("ouputPickle.pkl")

        # open pickle file
        dbfile1 = open('ouputPickle.pkl', 'ab')

        # save pickle data
        pickle.dump(json_object, dbfile1)                     
        dbfile1.close()
    
    data = {}
    data["status"] = 1
    json_data = json.dumps(data)
    json_object = json.loads(json_data) 
    return(json.dumps(json_object, indent = 1))


if __name__ == '__main__':
    app.run(host="0.0.0.0")
        
        